In [ ]:
# nome do experimento
proj = 'adults_FULL'

# delta do IQR para detecção de outliers
delta = 0


# tipo de espaço latente. 
# Se 'dense', tenderá a ser menor que tamanho dos dados originais. 
# Se "sparse", pode ser maior que o tamanho dos dados originais
center_type = 'dense'

# Número de interações do grid-search
iterations = 200

# Quantidade máxima de hidden layers no Encoder e Decoder
hidden = 1

# 01 - Imports

## 01.A - Installing Required Packages

## 01.B - Loading Packages

In [ ]:
import warnings

warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA

from scipy.stats import ks_2samp,wasserstein_distance

import pickle
import numpy as np
import pandas as pd
import time
from functools import partial

import matplotlib.pyplot as plt
import seaborn as sns
from plotly import tools
import plotly.graph_objects as go

import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras import applications
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.keras.losses import mse, binary_crossentropy
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Flatten, Conv1D

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval

## 01.C - Custom Classes and Functions

In [ ]:
# Custom Classes and Functions
from classes import xplor, LowerCase,MissingValue, Outliers, CharEncoder, ScalingTreatmeant, VariationalAutoencoder, compare_metric

In [ ]:
def build_model(params):
   
    l = [params[s] for s in params.keys() if s.startswith('unit')]

    return VariationalAutoencoder(
        layers_= l +  [params['center']],
        activations=[params['activation']]*len(l) + [params['center_activation']],
        final_act=params['final_activation'],
        loss="mean_squared_error",
        optimizer="adam",
        learning_rate = params['learning_rate']
    )

# 02 - Data Reading

In [ ]:
#od.download("https://www.kaggle.com/rameshmehta/credit-risk-analysis/version/1")
data = pd.read_csv("adult.csv", header = None)
data.columns = ['age','workclass','fnlwgt','education','education_num','marital_status','occupation','relationship','race','sex','capital_gain','capital_loss','hours_per_week','native_country', 'salary']

In [ ]:
data.shape

In [ ]:
data.head()

# 03 - Data Cleansing

In [ ]:
# Criando classe para fazer limepza dos dados de forma automatizada
# Essa classe consegue analisar e identificar variáveis com baixa qualidade em relação a valores nulos,
# variáveis categóricas com muitas variáveis, variáveis do tipo data, variáveis com altíssima variância 
# e variáveis com variância nula
xp = xplor(data)

In [ ]:
# Checando variáveis nulas.
# É realizado um gráfico de pareto para a % de nulos em cada variável.

# De acordo com o parâmetro 'level', serão selecionadas (para exclusão) as variaveis que 
# ultrapassarem o valor desse parâmetro. Ou seja, nesse acso todas as variáveis com mais de 50%
# de nulos serão selecionadas para exclusão

# Foram encontradas 21 variáveis.

xp.check_nulls(level = 0, select = True)
print(xp.nulls)

In [ ]:
# Este próximo método visa identificar variáveis categóricas que possuem uma quantidade alta de valores únicos,
# Neste exemplo, toda variável categórica com mais de 20 categrias distintas será selecionada para exclusão.

# Neste caso foram encontradas 10 variáveis.

# O Gráfico de pareto é mostrado para ajudar na identificação visual

xp.check_unique_objects(level_unique = 50,select = True)

In [ ]:
# O próximo método visa identificar variáveis do tipo data/
# Para este experimento, essas variáveis serão excluídas.

# Neste exemplo, foram encontradas 5 variáveis

xp.check_dates(select = True)

In [ ]:
# Neste último método, o objetivo é encontrar variáveis que possuam uma variância normalizada muito alta ou nula.
# O interessante foi verificar que ela se mostrou últi para encontrar 
# as duas colunas relacionadas ao ID (com altíssima variância) e uma coluna sem variância.

xp.check_var(select = True)

In [ ]:
# Para finalizar esse processo, é executado o método 'clean_data',
# que vai pegar todas as variáveis identificadas nos métodos anteriores e vai excluí-las da base final

new_df = xp.clean_data()

In [ ]:
# Com isso, foram excluídas 34 variáveis.
# A base final agora possúi 39 variáveis.
new_df.shape

# 04 - Data Treatmeant Pipeline (Pre-Processing)

In [ ]:
new_df.columns

In [ ]:
new_df.drop(['education_num'], axis = 1, inplace = True)

In [ ]:
# Para o pré-processamento dos dados, estou utilizando o método Pipeline do scikit-learn
# Este método é interessante pois podemos encadear vários pré-processamentos de dados e criar
# um único objeto que irá realizar todos os steps encadeados.

# Neste caso, foi-se construído um objeto que realizará 5 pré-processamentos:
# - Transformação de todos as variáveis categóricas para lower case
# - Input de missing values para numéricas e categóricas
# - Identificação e exclusão de outliers (apenas amostra treinamento)
# - Aplicação do método OneHotEncoder nos dados categóricos
# - Ajsute range dos dados para que eles fiquem entre 0 e 1.

#df_new = new_data.drop(['cus_cust_id'], axis = 1).copy()

if delta == 0:
    my_pipe = Pipeline([ ('LowerCase',LowerCase())
#                     ,('MissingValue',MissingValue(num_value = 'value', value_num = 0, obj_value = 'NULL'))
#                     ,('Outliers',Outliers(level = 1.5))
                    ,('CharEncoder',CharEncoder(methods = 'onehot'))
                    ,('ScalingTreatmeant',ScalingTreatmeant())
                   ])
else:
    my_pipe = Pipeline([ ('LowerCase',LowerCase())
            #            ,('MissingValue',MissingValue(num_value = 'value', value_num = 0, obj_value = 'NULL'))
                        ,('Outliers',Outliers(level = delta))
                        ,('CharEncoder',CharEncoder(methods = 'onehot'))
                        ,('ScalingTreatmeant',ScalingTreatmeant())
                       ])

In [ ]:
my_pipe

In [ ]:
# Ao aplicar o .fit(), a pipeline irá aprender diversos aspectos dos dados mas não irá alterá-los ainda.
my_pipe.fit(new_df.copy())

# 05 - Train Test Split + Transformation

In [ ]:
# Criação da amostra treino e teste.
# Nesse caso existe apenas as bases 'X', não sendo necessário a base 'y', pois o input é o próprio target.
# O objetivo do processo é replicar exatamente os dados de entrada.

X_train, X_test = train_test_split(new_df.copy(), test_size=0.2, random_state=42)

# Aqui estamos aplicando o .transform() doa pipeline treinada.
# Aqui os dados sofrerão as transformações necessárias para ficarem prontos para o treinamento
# da rede neural.

# Estou mudando um pequeno parâmetro da pipeline para ter a exclusão dos outliers para a amostra treino
if delta > 0:
    my_pipe.set_params(Outliers__train=True)
X_train_t = my_pipe.transform(X_train.copy())

# Estou mudando um pequeno parâmetro da pipeline para não ter a exclusão dos outliers na amostra teste
if delta > 0:
    my_pipe.set_params(Outliers__train=False)
X_test_t = my_pipe.transform(X_test.copy())

In [ ]:
import joblib
joblib.dump(my_pipe, 'pipeline_'+proj+'.sav')

In [ ]:
print(X_train_t.shape)
print(X_test_t.shape)


# 06 - Models

In [ ]:
# para realização do meu primeiro experimento, necessito escolher dois parâmetros:
# 1) Como será o centro do meu autoencoder: pode ser 'dense' ou 'sparse'. 
#    O 'dense' limita o centro a ser no máximo 40% do tamanho original dos dados de input, 
#    enquanto o 'sparse' retira esse limitante e deixa livre para o modelo escolher o tamanho
#    que for melhor para garantir o melhor resultado.

# 2) Quantidade de iteraçoes do Tree of Panzer Element, ou seja, quantas combinações de parâmetros serão utilizadas.
#    Como teste, vc pode colocar esse valor como 1. Os modelos que eu treinei serão carregados mais à frente

# dimensão do input
base_dim = X_train_t.shape[1]

# Ajuste do tamanho máximo e mínimo de cada layer, para que a arquitetura final tenha formato de ampulheta
# conforme figura mostrada acima
ls = np.linspace(1, 10, 10 * hidden) / 10

lat_ls = ls[0 : int(len(ls) * 0.4)]
hid_aux = ls[int(len(ls) * 0.4) :]

hid_aux = list(reversed(hid_aux))
hid_ls = []
for i in range(0, len(hid_aux), int(len(hid_aux) * (1 / hidden))):
    hid_ls.append(hid_aux[i : i + int(len(hid_aux) * (1 / hidden))])

# Criação do espaço de possibilidades dos hyper-parametros.
# Os hiperparametros são:
# - Quantidade de Nodes em cada layer
# - Funções de ativação na camada final do enconder
# - Funções de ativação na camada final do deconder
# - Funções de ativação nos hidden layers

space = {
    "units"
    + str(i): hp.choice(
        "units" + str(i), [int(base_dim * (p)) for p in hid_ls[i]]
    )
    for i in range(len(hid_ls))
}
if center_type == "sparse":
    print([int(base_dim * (p)) for p in ls[0:-1:hidden]])
    center = {
        "center": hp.choice(
            "center", [int(base_dim * (p)) for p in list(ls[0:-1:hidden]) + [1, 1.1,1.2,1.5,2] ]
        )
    }
elif center_type == "dense":
    print([int(base_dim * (p)) for p in lat_ls])
    center = {
        "center": hp.choice(
            "center", [int(base_dim * (p)) for p in lat_ls]
        )
    }

others = {
    "optimizer": hp.choice("optimizer", ["adam"]),
    "activation": hp.choice("activation", ["relu", "tanh", "selu","sigmoid"]),
    "center_activation": hp.choice(
        "center_activation", ["sigmoid", "linear", "tanh", "relu","selu"]
    ),
    "final_activation": hp.choice(
        "final_activation", ["sigmoid", "tanh"]
    ),
    "learning_rate": hp.choice(
        "learning_rate", [0.1, 0.05,0.01,0.005,0.001,0.0001,0.00001]
    ),
}
space.update(center)
space.update(others)

In [ ]:
# Função para criar a arquitetura do modelo, treiná-lo e retornar o erro da amostra validação
# Esse erro será utilizado pelo Tree of Parzen Element para 'chutar' uma próxima combinação
def f_nn(params):
    # Criação da arquitetura: quantidade de layers, funções de ativação, função de perda e otimizador
    ae = VariationalAutoencoder(
        layers_=[params[u] for u in unitss] + [params['center']],
        activations=[params['activation']] * len(unitss) +  [params['center_activation']],
        final_act=params['final_activation'],
        loss="binary_crossentropy",
        optimizer="adam",
        learning_rate = params['learning_rate']
    )

    # treino do modelo
    ae.fit(

        X_train_t,
        Y=None,
        epochs=500,
        batch_size=300,
        shuffle=True,
        verbose=0,
        save=0,
        proj=0,
    )
    
    # Captura do valor da loss da validação da última iteração do modelo
    acc = ae.autoencoder.history.history["val_loss"][len(ae.autoencoder.history.history["val_loss"]) - 1]
    return {"loss": acc, "status": STATUS_OK}
     
# Criação do objeto hyperopt que realizará o grid-search
trials = Trials()

algo = partial(
    tpe.suggest,
    n_startup_jobs=int(iterations * 0.3),
    gamma=0.25,
    n_EI_candidates=24,
)

best = fmin(f_nn, space, algo=algo, max_evals=iterations, trials=trials)

# Ao final, a função fmin retornará o hyper-parametros que obtiveram o melhor resultado
best_p = space_eval(space, best)

# Salvando o melhor resultado num arquivo teste
# O arquivo orginal será carregado mais abaixo
with open( proj + "_sparse_1hl_params_model.pkl", "wb") as f:
            pickle.dump(space_eval(space, best), f)
best_p